<h3> Import packages </h3>

In [78]:
import pandas as pd
import numpy as np
import os
import time
import geocoder
from geopy.geocoders import Nominatim

<h3> Read in data </h3>

Bigfoot data came in two dataframes; location and reports. UFO data came in one dataframe. All three dataframes are very large, so I ran them in separate cells to avoid any issues

In [79]:
BF_Locations = pd.read_csv(os.path.join("https://download.data.world/s/a5gdzsmxenwcwqmnbjaaizbfk4uea3"))

In [80]:
BF_Reports = pd.read_csv(os.path.join("https://download.data.world/s/td52m5guv2cpruwjo6pr2cd3fbk5cx"))

In [81]:
UFO = pd.read_csv(os.path.join("https://download.data.world/s/lobqhnvklhbppb4ayke4b23lsupg57"))

<h3> Cleaning Bigfoot Data </h3>

Change column name in bigfoot locations dataframe, in order to merge both bigfoot dataframes into one.

In [82]:
BF_Locations = BF_Locations.rename(columns={'number': 'report_number'})

Merged both bigfoot dataframes to make one cohesive dataframe.

In [83]:
BF = pd.merge(BF_Locations, BF_Reports, how='left', left_on='report_number', right_on='report_number')

Split 'title' column. Title column had both the report and report number. Report number was already given in another cell, so I split the column to obtain the report by itself.

In [84]:
BF[['report_number2', 'report']] = BF.pop('title').str.split(pat=':', n=1, expand=True)

Reformatting the 'timestamp' column, in order to separate the date and time into two columns.

In [85]:
BF['timestamp'] = pd.to_datetime(BF['timestamp'], format = "%Y-%m-%dT%H:%M:%S.%f")

Splitting 'timestamp' column.

In [86]:
BF['report_date'] = BF['timestamp'].dt.date
BF['time'] = BF['timestamp'].dt.time

Removing unnecessary columns in bigfoot dataframe.

In [87]:
BF = BF.drop(['timestamp', 'year', 'season', 'report_class', 
              'month', 'date', 'a_g_references', 'time', 'report_number2', 
              'report_number', 'classification', 'county', 'nearest_town', 
              'nearest_road', 'also_noticed', 'other_witnesses', 'other_stories', 
              'time_and_conditions', 'environment', 'location_details', 'observed'], axis=1)

Changing 'report date' column to datetime format.

In [88]:
BF['report_date'] = pd.to_datetime(BF['report_date'], errors='coerce')

Cleaning column names

In [89]:
BF = BF.rename(columns={'report_date': 'date'})

<h3> Resizing Data </h3>

Due to how large both the UFO and bigfoot dataframes are, I will be using only the first 1,000 rows for the project.

In [90]:
BF = BF[:1000]
UFO = UFO[:1000]

<h3> Cleaning UFO Data </h3>

Removing unnecessary columns.

In [91]:
UFO = UFO.drop(['stats', 'report_link', 'text', 'shape', 'duration', 'posted'], axis = 1)

Cleaning column names.

In [92]:
UFO = UFO.rename(columns={'summary': 'report'})

For this project I will only be analyzing sightings in the United States. Here I am removing all other countries.

In [93]:
UFO = UFO.loc[UFO["country"] == "USA"]

Splitting 'date_time' column.

In [94]:
UFO[['date', 'time']] = UFO.pop('date_time').str.split(pat=' ', n=1, expand=True)

Changing 'date' column to datetime format.

In [95]:
UFO['date'] = pd.to_datetime(UFO['date'], errors='coerce')

Dropping null values.

In [96]:
UFO = UFO[UFO['date'].notna()]
UFO = UFO[UFO['city'].notna()]
UFO = UFO[UFO['state'].notna()]

<h3> Cleaning for Geocoding </h3>

The goal for this project was to analyze a map of the sighting reports. In order to map the reports I wanted to find the coordinates of the UFO report locations. Bigfoot coordinates were already provided. In order to acquire the longitude and latitude for the UFO reports I used geocoding. My process is shown below.

The data from the 'city' and 'state' columns needed to be thoroughly cleaned, or else the geocoding would not work. I started by capitalizing the city names.

In [97]:
UFO.city = UFO.city.str.title()

Next the city and state names needed to be cleaned. There was quite a few names that were misspelled, inaccurate, or just vague descriptions. Here I am correcting the errors.

In [98]:
UFO = UFO.replace({'city':{
    'St Augustine Pass Las Cruces':'San Augustin',
    'Kentucky (Somewhere In North Central Ky On I-65)': 'Covington',
    'Airmont, (Formerly Tallman)': 'Tallman',
    'Panther Creek State Park, Close To Morristown, Tennessee': 'Morristown',
    'Washington Dc (Suitland, Md)': 'Suitland',
    'Saguaro Lake (Mesa)': 'Mesa',
    'Grand Rapids - Godwin Heights': 'Grand Rapids',
    'Terre Haute (Just E Of; On Interstate 70)': 'Terre Haute',
    'Near Jemez': 'Jemez',
    ' Eldorado Hills-Placerville (Hwy 50': 'Placerville',
    'Odessa (Starkey Ranch)': 'Odessa',
    'Staton Island': 'New York City',
    'Zoar (Small Town In America)': 'Zoar',
    'Lordsburg And Silver City (Between)': 'Lordsburg',
    'Swayze': 'Swayzee',
    'Brainerd/Gull Lake': 'Brainerd',
    'Chrystal River': 'Crystal River',
    'Two Or Three Miles Northwest Of Detroit Metro Airport.': 'Detroit',
    'Seaford, Long Island': 'Long Island',
    'Bosie': 'Boise',
    'Seiverville': 'Sevierville',
    'Towsan': 'Towson',
    'Small Town Off The I-40 Not Sure Which)': 'Null',
    'Unkone': 'NaN',
    'Unsure Exactly': 'NaN',
    'Port St. Lucie': 'Port Saint Lucie',
    'De Funiak Springs': 'DeFuniak Springs',
    'South Ozone Park, Queens, Nyc': 'New York City',
    'St. Louis': 'Saint Louis',
    'Forest Hills (Queens; Nyc)': 'New York City',
    'Huntsville Alabama': 'Huntsville',
    'Idaho National Forest': 'NaN',
    'Ft Worth': 'Fort Worth',
    'Sellersville/Perkasie, Pa': 'Sellersville',
    'Mt. Vernon': 'Mount Vernon',
    'Rio Ranco': 'Rio Rancho',
    'Forest Hills (Queens)': 'New York City',
    'Lauderdale By The Sea Florida': 'Fort Lauderdale',
    'Newport Rhode Island': 'Newport',
    'Daytona Beach Shores': 'Daytona Beach',
    'Ft Pierce': 'Fort Pierce',
    'Road Above Stinson Beach': 'Stinson Beach',
    'Near The Wisconsin Dells': 'NaN',
    'South Of Zortman': 'NaN',
    'Pompano Beach Pompano': 'Pompano Beach',
    'Lakewood/Edgewater': 'Lakewood',
    'Carrabelle Florida': 'Carrabelle',
    'Interstate 80': 'NaN',
    'Blaine-Outside City Limits': 'Blaine',
    'Batesville And Floral': 'Batesville',
    'Clarkston, Washington': 'Clarkston',
    'Port St Lucie': 'Port Saint Lucie',
    'Between Faster,Al And Tuscaloosa, Al': 'Tuscaloosa',
    'St. Johns Du Quoin Address': 'Saint Johns',
    'St. George': 'Saint George',
    '21-Minutes, Flight Time, North Of Phoenix': 'Phoenix',
    'Boston Area': 'Boston',
    'Near Muhlenburg Airport': 'NaN',
    'Taylorsville, Salt Lake, Utah': 'Salt Lake City',
    'Orem, Provo': 'Orem',
    'N Las Vegas': 'Las Vegas',
    'Lafollette And Powell': 'LaFollette',
    'St. Joseph': 'Saint Joseph',
    'Rosalia, St. John, Colfax Area Of The Palouse': 'Rosalia',
    'Wentzville,': 'Wentzville',
    'Columbus, Ohio And Surrounding Areas': 'Columbus',
    'Monterey Bay/Fort Ord Rifle Range': 'Monterey Bay',
    'Woodberry Forest Preparatory School, Va': 'NaN',
    'Los Angeles (Over I-10, On Most Rural Bayou Stretch)': 'Los Angeles',
    'California Valley (Cattle Country, Not Too Distant Airbase)': 'California Valley',
    'Bayfield (Near, Lake Superior, South Shore)': 'Bayfield',
    'Between Tacoma And Gig Harbor, Wa': 'NaN',
    '20,000 Feet Over Trout Lake, Wa Looking Northwest': 'NaN',
    'Thomasville (Usaf Long Range Radar Facility)': 'Thomasville',
    'Myrtle Beach, Horry County South Carolina': 'Myrtle Beach',
    'Olyimpa': 'Olympia',
    'Vancounver': 'Vancouver',
    'Mckinelyville': 'Mckinleyville'
}})

UFO = UFO.replace({'state':{
    'Washington, DC': 'MD',
    'Cornwall': 'NaN',
    '\nMA': 'MA'
}})

There were many entries that were too vague to assign a city, or city names that were actually mistakenly labeled for the incorrect country. Here I am removing those specific rows.

In [99]:
UFO = UFO.drop(labels=[44, 65, 141, 536, 595, 106, 200, 449, 546, 616, 740, 906, 990, 972], axis=0

Below is the cleaned UFO dataframe before geocoding.

In [100]:
UFO.head()

,city,state,country,report,date,time
0,Sisters,OR,USA,Long and narrow illuminated craft flying high ...,2021-05-15,22:36
1,Sarasota,FL,USA,String of lights,2021-05-10,22:00
2,Cleveland,TN,USA,30+ lights in a line.,2021-05-05,22:35
3,Galway,NY,USA,I was standing at my kitchen sink and looked u...,2020-10-16,18:25
4,Downingtown,PA,USA,Multiple slow moving craft seen all over the s...,2021-06-18,01:00


<h3> Geocoding </h3>

Due to the time it takes for the geocoding to process, I will be showing the code used for the geocoding, but then exporting the results. The below code was ran to obtain the longitude and latitude results, then commented out so users can simply import the results (which is done below) or can uncomment the below code to run it themselves. For me the process took approximately seven minutes, but that can vary per user.

Connecting to API

In [48]:
"""geolocator = Nominatim(user_agent="MyApp")
query= 'Louisville, Ky'
results = geolocator.geocode(query)
print(results)"""

Louisville, Jefferson County, Kentucky, United States


Conducting the geocoding. City and state columns were acquired to obtain longitude and latitude coordinates, and then assigned to appropriate columns.

In [49]:
"""lat_list= []
long_list= []

incorrect_cities = []

for index, row in UFO.iterrows():
    
    city = row['city']
    state = row['state']
    query = str(city) + ',' + str(state)
    
    results = geolocator.geocode(query)
    
    if results:
        lat = results.latitude
        long = results.longitude
    
    else:
        incorrect_cities.append(query)
    
    lat_list.append(lat)
    long_list.append(long)
    
    time.sleep(0.1)

UFO['latitude'] = lat_list
UFO['longitude'] = long_list"""

If any city names were incorrect, they were shown below. This is how I was able to replace the city names above.

In [25]:
"""incorrect_cities"""

[]

New UFO dataframe with added longitude and latitude columns.

In [51]:
"""UFO.head()"""

,city,state,country,report,date,time,latitude,longitude
0,Sisters,OR,USA,Long and narrow illuminated craft flying high ...,2021-05-15,22:36,44.290948,-121.549252
1,Sarasota,FL,USA,String of lights,2021-05-10,22:00,27.336581,-82.530855
2,Cleveland,TN,USA,30+ lights in a line.,2021-05-05,22:35,35.159518,-84.876611
3,Galway,NY,USA,I was standing at my kitchen sink and looked u...,2020-10-16,18:25,43.018686,-74.031516
4,Downingtown,PA,USA,Multiple slow moving craft seen all over the s...,2021-06-18,01:00,40.006496,-75.703274


Exporting the latitude and longitude columns into a json file.

In [52]:
"""UFO.loc[:, ['latitude', 'longitude']].to_json('coords.json')"""

<h3> Reading in Geocoding </h3>

Below the user is able to read in the exported json we did above. 

In [76]:
coords = pd.read_json("coords.json")
UFO.loc[:, ['latitude', 'longitude']] = coords.loc[:, ['latitude', 'longitude']]

UFO dataframe. Should be the same as the dataframe shown after the geocoding.

In [77]:
UFO.head()

,city,state,country,report,date,time,latitude,longitude
0,Sisters,OR,USA,Long and narrow illuminated craft flying high ...,2021-05-15,22:36,44.290948,-121.549252
1,Sarasota,FL,USA,String of lights,2021-05-10,22:00,27.336581,-82.530855
2,Cleveland,TN,USA,30+ lights in a line.,2021-05-05,22:35,35.159518,-84.876611
3,Galway,NY,USA,I was standing at my kitchen sink and looked u...,2020-10-16,18:25,43.018686,-74.031516
4,Downingtown,PA,USA,Multiple slow moving craft seen all over the s...,2021-06-18,01:00,40.006496,-75.703274


Dropping rows not needed for Tableau mapping.

In [26]:
UFO = UFO.drop(['time', 'city', 'country'], axis = 1)

<h3> Exporting Cleaned Dataframes </h3>

Tableau is unable to read in Jupyter Notebook files. Below I am exporting both dataframes into one excel file, but on separate sheets, in order to then read in to Tableau.

In [29]:
with pd.ExcelWriter('UFO_BF.xlsx') as writer:
    BF.to_excel(writer, sheet_name='BF')
    UFO.to_excel(writer, sheet_name='UFO')